In [32]:
import numpy as np

In [33]:
import pandas as pd

In [34]:
%%writefile test_utility.py
import logging
import os
from pyaml import yaml
import pandas as pd
import gc
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLerror as exc:
            logging.error(exc)
            
def col_header_val(df, table_config):
    df.columns = df.columns.str.lower()
    expected_col = list(map(lambda x: x.lower(), table_config['columns']))
    expected_col.sort()
    df.columns = list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis = 1)
    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print("columns name and column length validation passed")
        return 1
    else:
        print("columns name and column length validation failed")
        missmatched_column_file = list(set(df.columns).difference(expected_col))
        print("The following files are not found in the YAML file", missmatched_column_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("The following files are not uploaded in the file upload", missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0
    

Overwriting test_utility.py


In [35]:
%%writefile file.yaml
file_type: csv
dataset_name: test_data
file_name: animelists_cleaned
table_name: edsurv
inbound_delimiter: ","
outbound_delimeter: "|"
skip_leading_rows: 1
columns:
    - username
    - anime_id
    - my_watched_episodes
    - my_start_date
    - my_finish_date
    - my_score
    - my_status
    - my_rewatching
    - my_rewatching_ep
    - my_last_updated
    - my_tags

Overwriting file.yaml


In [36]:
# Read config file
import test_utility as util
config_data = util.read_config_file('file.yaml')

In [6]:
%%time
#test 1 pandas

df = pd.read_csv('datasets/animelists_cleaned.csv/animelists_cleaned.csv')

CPU times: total: 1min 24s
Wall time: 1min 26s


In [8]:
import test_uitlity as util

util.col_header_val(df, config_data)

columns name and column length validation passed


1

In [9]:
from dask.distributed import Client
client = Client(memory_limit = '4GB')
client


C:\Users\aaric\AppData\Local\Programs\Python\Python310\lib\site-packages\distributed\node.py:181: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50153 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:50153/status,
Dashboard: http://127.0.0.1:50153/status,Workers: 4
Total threads: 16,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50154,Workers: 4
Dashboard: http://127.0.0.1:50153/status,Total threads: 16
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:50184,Total threads: 4
Dashboard: http://127.0.0.1:50186/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:50158,


In [23]:
%%time
import dask.dataframe as dd

df = dd.read_csv('datasets/animelists_cleaned.csv/animelists_cleaned.csv', dtype={'my_tags': 'object'})
df.compute()

CPU times: total: 50.2 s
Wall time: 1min 15s


,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,2013-03-03 10:52:53,NaN
1,karthiga,59,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-10 13:54:51,NaN
2,karthiga,74,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-04-27 16:43:35,NaN
3,karthiga,120,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-03 10:53:57,NaN
4,karthiga,178,26,0000-00-00,0000-00-00,7,2,0.0,0,2013-03-27 15:59:13,NaN
...,...,...,...,...,...,...,...,...,...,...,...
888908,Yokonightcore,15611,48,2014-00-00,0000-00-00,9,1,NaN,0,2015-09-07 17:33:03,NaN
888909,Yokonightcore,27815,22,0000-00-00,0000-00-00,9,1,NaN,0,2015-09-07 17:32:05,NaN
888910,wargod,5945,39,0000-00-00,0000-00-00,8,2,0.0,0,2010-03-29 04:24:12,NaN
888911,JMc_SetoKai_LoVe,1316,52,0000-00-00,0000-00-00,9,2,NaN,0,2009-12-23 05:45:14,NaN


In [39]:
df

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
npartitions=35,,,,,,,,,,,
,object,int64,int64,object,object,int64,int64,float64,int64,object,object
,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...


In [40]:
%%time
#test 2 read using the config file

file_type = config_data['file_type']
source_file = "datasets/animelists_cleaned.csv/" + config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file, config_data['inbound_delimiter'])

CPU times: total: 1min 39s
Wall time: 1min 42s


In [ ]:
df.to_csv("test_data.csv.gz", 
           index=False, 
           compression="gzip")